In [1]:
import numpy as np
from statsmodels.tsa.stattools import adfuller,kpss
import pandas as pd
from matplotlib import pyplot as plt
import torch
import gpytorch
import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys
from tigramite import plotting as tp
from tigramite.lpcmci import LPCMCI
#from tigramite.pcmci import PCMCI
#from tigramite.independence_tests.parcorr import ParCorr
#from statsmodels.tools.sm_exceptions import InterpolationWarning
import warnings
from tigramite.lpcmci import LPCMCI
import seaborn as sns
#from tigramite.independence_tests.gpdc import GPDC
#from tigramite.independence_tests.gpdc_torch import GPDCtorch
from tigramite.independence_tests.cmiknn import CMIknn
# from tigramite.independence_tests.cmisymb import CMIsymb

In [2]:
dat = pd.read_csv('dv_norm_loc.csv', header = None)
print(dat.head())

     0       1       2       3       4       5       6       7       8    \
0  24.94  24.347  24.347  23.752  23.752  23.153  23.153  22.555  22.555   
1  14.95  14.517  14.517  14.085  14.085  13.653  13.653  13.224  13.224   
2  23.43  22.732  22.732  22.058  22.058  21.409  21.409  20.777  20.777   
3  12.31  12.218  12.218  12.162  12.162  12.136  12.136  12.123  12.123   
4  30.45  29.788  29.788  29.125  29.125  28.462  28.462  27.805  27.805   

      9    ...     134     135     136     137     138     139     140  \
0  21.965  ...   2.315   2.027   2.027   1.740   1.740   1.450   1.450   
1  12.796  ...   8.340   8.649   8.649   8.971   8.971   9.311   9.311   
2  20.153  ...   2.322   2.586   2.586   2.854   2.854   3.126   3.126   
3  12.107  ...  13.060  13.068  13.068  13.058  13.058  13.037  13.037   
4  27.143  ...   5.139   4.986   4.986   4.857   4.857   4.751   4.751   

      141     142     143  
0   1.159   1.159   0.873  
1   9.665   9.665  10.035  
2   3.413   3.

In [3]:
# Load Data, Handle Missingness:

data = torch.load(f"/home/gnicolaou/tigramite/tutorials/causal_discovery/combined_tensor.pt", map_location=torch.device('cpu'))

# Number of observed variables
#N = data.shape[1]
data = data.numpy()
data = pd.DataFrame(data)
print(data.shape)

(44064, 8)


In [4]:
array_306_144 = dat.values.flatten()
data[8] = array_306_144
print(data.head())

            0         1           2          3         4          5  \
0   66.216728  0.078297  292.137177  25.787775  0.000160  10.428891   
1  200.731400  0.007993  295.146606  23.221931  0.000240   8.244879   
2   24.651501  0.013119  292.934418  23.455170  0.000224  12.020223   
3  159.606262  0.001872  294.594208  17.631453  0.007888   2.550540   
4   12.181140  0.003811  291.437225  21.168480  0.000320  10.262062   

              6         7       8  
0 -1.708685e-05  0.902100  24.940  
1 -1.117597e-06  0.933838  24.347  
2  1.190958e-06  0.901123  24.347  
3 -6.610977e-07  0.575684  23.752  
4  1.108296e-05  0.645508  23.752  


In [5]:
dat = data.values

# empty list to store the modified columns to separate trajectories
modified_columns = []

for col in range(dat.shape[1]):
    column_data = dat[:, col]
    
    # insert 999 values between every 144 elements
    modified_column = np.insert(column_data, np.arange(144, column_data.size, 144), 999)
    
    # append the modified column to the list
    modified_columns.append(modified_column)

modified_data = np.column_stack(modified_columns)

dat = modified_data # handle missingness
n_a_n = np.isnan(dat).any(axis=1)
dat[n_a_n] = 999
#dat = dat[0:1440,:]

# initialize dataframe object, specify variable names
var_names = ['Nd','Pr','sst','lts','fth','ws','div','cf', 'loc']
dataframe = pp.DataFrame(dat, var_names=var_names, missing_flag = 999)

In [14]:
cmiknn = CMIknn(significance='shuffle_test', knn=0.1, shuffle_neighbors=5, transform='ranks', sig_samples=200)
link_assumptions = {j:{(i, -tau):'' for i in range(9) for tau in range(2) if (i, -tau) != (j, 0)} 
                            for j in range(9)}

link_assumptions[0][(1, 0)] = '<?-' #Nd is an ancestor of P
link_assumptions[1][(0, 0)] = '-?>'
link_assumptions[7][(1, 0)] = '<?-'# CF is an ancestor of precipitation
link_assumptions[1][(7, 0)] = '-?>'
link_assumptions[0][(0, -1)] = '-?>' #Nd at lag t-1 is an ancestor of Nd
link_assumptions[0][(1, -1)] = '-?>' #P at lag t-1 is an ancestor of Nd
link_assumptions[7][(1, -1)] = '-?>' #P at lag t-1 is an ancestor of CF

link_assumptions[7][(0, -1)] = 'o?>' #Nd at lag t-1 is an ancestor of CF
link_assumptions[1][(0, -1)] = 'o?>' #Nd at lag t-1 is an ancestor of P
link_assumptions[1][(7, -1)] = 'o?>' #CF at lag t-1 is an ancestor of P
link_assumptions[7][(7, -1)] = 'o?>' #CF at lag t-1 is an ancestor of CF

for j in range(2,7): 
    link_assumptions[j][(0, 0)] = '<?-' #meterological variables are ancestor of aerosol
    link_assumptions[0][(j, 0)] = '-?>'
    link_assumptions[j][(7, 0)] = '<?-' #meteorological variables are ancestor of cloud fraction
    link_assumptions[7][(j, 0)] = '-?>'
    link_assumptions[j][(8,0)] = 'o?o' #meteorological variables has a link wth location
    link_assumptions[8][(j,0)] = 'o?o'

link_assumptions[0][(8, 0)] = 'o?o' #Nd has any link with location contemporaneously
link_assumptions[8][(0, 0)] = 'o?o' 
link_assumptions[1][(8, 0)] = 'o?o' #P has any link with location contemporaneously
link_assumptions[8][(1, 0)] = 'o?o' 
link_assumptions[7][(8, 0)] = 'o?o' #CF has any link with location contemporaneously
link_assumptions[8][(7, 0)] = 'o?o' 
link_assumptions[0][(8, -1)] = 'o?>' #Nd has any link with location at lag t - 1
link_assumptions[8][(0, -1)] = 'o?>' 
link_assumptions[1][(8, -1)] = 'o?>' #P has any link with location at lag t - 1
link_assumptions[8][(1, -1)] = 'o?>' 
link_assumptions[7][(8, -1)] = 'o?>' #CF has any link with location at lag t - 1
link_assumptions[8][(7, -1)] = 'o?>' 
print(link_assumptions)

{0: {(0, -1): '-?>', (1, 0): '<?-', (1, -1): '-?>', (2, 0): '-?>', (2, -1): '', (3, 0): '-?>', (3, -1): '', (4, 0): '-?>', (4, -1): '', (5, 0): '-?>', (5, -1): '', (6, 0): '-?>', (6, -1): '', (7, 0): '', (7, -1): '', (8, 0): 'o?o', (8, -1): 'o?>'}, 1: {(0, 0): '-?>', (0, -1): 'o?>', (1, -1): '', (2, 0): '', (2, -1): '', (3, 0): '', (3, -1): '', (4, 0): '', (4, -1): '', (5, 0): '', (5, -1): '', (6, 0): '', (6, -1): '', (7, 0): '-?>', (7, -1): 'o?>', (8, 0): 'o?o', (8, -1): 'o?>'}, 2: {(0, 0): '<?-', (0, -1): '', (1, 0): '', (1, -1): '', (2, -1): '', (3, 0): '', (3, -1): '', (4, 0): '', (4, -1): '', (5, 0): '', (5, -1): '', (6, 0): '', (6, -1): '', (7, 0): '<?-', (7, -1): '', (8, 0): 'o?o', (8, -1): ''}, 3: {(0, 0): '<?-', (0, -1): '', (1, 0): '', (1, -1): '', (2, 0): '', (2, -1): '', (3, -1): '', (4, 0): '', (4, -1): '', (5, 0): '', (5, -1): '', (6, 0): '', (6, -1): '', (7, 0): '<?-', (7, -1): '', (8, 0): 'o?o', (8, -1): ''}, 4: {(0, 0): '<?-', (0, -1): '', (1, 0): '', (1, -1): '', (2, 

In [15]:
lpcmci_loc = LPCMCI(
    dataframe=dataframe, 
    cond_ind_test=cmiknn,
    verbosity=1)
results = lpcmci_loc.run_lpcmci(tau_max=2, pc_alpha=.05, link_assumptions = link_assumptions)
tp.plot_time_series_graph(graph=results['graph'],
                          val_matrix=results['val_matrix'], save_name = "cmiknn_w_loc.png")


Starting preliminary phase  1

Starting test phase

p = 0
